<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S31_vacuum_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install -U -q gtbook


Note: you may need to restart the kernel to use updated packages.


In [6]:
import gtsam
import gtbook

from gtbook import display
from gtbook.discrete import Variables
from gtbook import vacuum # various definitions for this chapter

VARIABLES = Variables()
def pretty(obj):
    return display.pretty(obj, VARIABLES)


# Modeling the State of the Vacuum Cleaning Robot

> We introduce discrete states as an abstraction of navigationg through continuous spaces.

<img src="Figures3/S31-iRobot_vacuuming_robot-04.jpg" alt="Splash image with 6 panels of a robot in different states" width="40%" align=center style="vertical-align:middle;margin:10px 0px">

In this chapter, we describe the state space for our vacuum cleaning robot.
Like the trash sorting robot, the vacuuming robot has a discrete state space.
However, unlike the trash sorting robot, transitions between arbitrary states is not possible for the vacuuming robot.
We will use a graph to encode this aspect of the the state space of the vacuuming robot.

## Defining the Robot's State

As described in Chapter 1, the representation of a robot's state should include all information that is necessary for the robot to act effectively in its environment to achieve its goals.  For a vacuum cleaning robot, this might include the exact location of the robot (e.g., x-y coordinates in a map of the house),
the heading direction of the robot,
whether the floor is carpeted,
or the location of any pets or children that might be moving throughout the house.

In this chapter, we will abstract away many of these details by making the following
assumptions:
- The robot can move in any direction, which means that its heading angle is not important.
- The robot is equipped with low-level navigation software that will allow it to move from one room to another, through doorways, etc. (though not with 100% reliability).
- The robot is equipped with path planning software to clean the floor in a particular room (e.g., execute random
motions, or follow a boustrophedon path).
- The robot is equipped with collision-avoidance software, so that it need not worry about the presence of obstacles (e.g., furniture, small dogs, or children).

Taken together, these assumptions allow us to abstract away all of the geometry of the vacuum cleaning problem,
and to **define the state of the robot as the room** of the house in which it is located.

As a running example, we will consider a simple house that
includes five rooms: living room, kitchen, office, hallway, and dining room.
The floor plan is illustrated 
below, in Figure <a href="#fig:Vacuum" data-reference-type="ref" data-reference="fig:House">1</a>.

<figure id="fig:House">
<img src="https://github.com/gtbook/robotics/blob/main/Figures3/S31-House.png?raw=1" alt="" />
<figcaption>The floor plan of a a house in which our hypothetical vacuum robot will operate.</figcaption>
</figure>

As for the trash sorting robot, we define the state to be a random variable with a prior probability distribution.
We denote the random state at time $t$ by $X_t$.

In this chapter, we will consider the case of **discrete time systems**.
In particular, rather than consider $t$ as a continuous variable, we will consider the system evolution
at a set of discrete moments in time, $t \in \{ t_1, t_2 \dots \}$.
This approach is appropriate for systems whose state changes qualitatively at distinct
moments of time (e.g., as when the robot moves from the living room to the kitchen),
rather than evolving continuously as time passes (e.g., the position of a robot that rolls along the floor).
While the physical vacuum cleaning robot rolls along the floor to reach its destination,
our representation of state includes only the room in which the robot is currently located.
Therefore, the only interesting moments in time are when the robot traverses from one room to the next, or -as we will see later in this chapter- when the robot executes an action or takes a sensor measurement.
To keep notation simple, we will let  $t \in \{ 1, 2 \dots \}$, rather than using the more general $t_i$ notation.
<!-- The concept of state is an abstraction, allowing robots to reason about what is happening to them and their environment as they act. This does not necessarily mean that we, as robot programmers, have no deeper knowledge of what is going on: we do know that a vacuuming robot operates in a two dimensional space, typically a single floor in a domestic residence. However, we might have programmed in "actions" that abstract to the notion of moving from room to room, and hence may only be interested in knowing which room the robot is in for the purposes of triggering those actions. Other actions that are useful but will not discuss further in this section might involve vacuuming the current room, terminating when it is done or when the robot is stuck. Rather, we will concentrate below on navigation actions, moving from room to room, knowing that it might be only a small part of the repertoire that a true vacuuming robot needs to be useful. -->

Since the robot will execute a sequence of actions over time, our prior distribution $P(X_1)$ will only encode knowledge about the initial state $X_1$ of the robot, and the probability distribution for future states must be determined using
probabilistic inference.
In this chapter, we will assume that the robot *always* starts out in the office, where its charging station is located,
i.e., $P(X_1 = Office) = 1.$ The following code does this:


In [8]:
# vacuum.rooms = ["Living Room", "Kitchen", "Office", "Hallway", "Dining Room"]
X = VARIABLES.discrete_series("X", [1], vacuum.rooms)
prior = gtsam.DiscreteDistribution(X[1], "0/0/1/0/0")
pretty(prior)


X1,value
Living Room,0
Kitchen,0
Office,1
Hallway,0
Dining Room,0


We use this discrete distribution to convey our knowledge that the robot *always* starts out in the office. We do this with $100\%$ certainty in this case. This use of probability values to indicate how certain we are about facts in the world is a "Bayesian" view, as we explain below.

## The State Space for our Vacuum Cleaning Robot

For the trash sorting robot, the state was defined as the category of the current item in the work space.
This category had no dependence on the category of the previous piece of trash, nor did it have any effect on the category
of the subsequent piece of trash. There were no constraints on the transition from the state at time $t$
to the state at time $t+1$.
Therefore, it was sufficient to merely enumerate the set of possible states; there
were no important relationships between states that required representation.

This is not the case for our vacuuming robot.
For example, as can be seen from
Figure <a href="#fig:Vacuum" data-reference-type="ref" data-reference="fig:House">1</a>,
if the robot is currently in the office, it cannot transition directly to the living room;
it must first transition to the hallway before it can transition to the living room.
For the vacuum cleaning robot, room adjacency is an important relationship, and therefore
it should be encoded into our representation.
This can be accomplished using a connectivity graph.
Each vertex of this graph represents a state (i.e., a specific room in the house), and two
vertices, $x_i, x_j$ are connected by an edge if and only if it the transition between
these two states is possible.  Since our robot can move in any direction, if it
can transition from $x_i$ to $x_j$, then it can also transition from $x_j$ to $x_i$
(as is the case, e.g., for $x_i = Hallway$ and $x_j = Dining Room$).
Therefore, we represent the state space by an *undirected* graph.
Taken together, the collection of states along with connectivity information is referred to as the **state space**,
which in this can be represented by the connectivity graph shown in 
Figure <a href="#fig:Vacuum" data-reference-type="ref" data-reference="fig:House-ss">2</a>.

<figure id="fig:House-ss">
<img src="https://github.com/gtbook/robotics/blob/main/Figures3/S31-house-state-space.png?raw=1" alt="" />
<figcaption>The state space corresponding
to the house above.</figcaption>
</figure>

## Bayesian vs. Frequentist

<figure id="fig:Bayesian">
<img src="https://github.com/gtbook/robotics/blob/main/Figures3/N3-Bayes.png?raw=1" style="width:14cm" alt="" />
<figcaption>The Reverend Thomas Bayes gave his name to associating probabilities with the strength of beliefs rather than a frequency of events, even though this seems to have been first introduced by Laplace.</figcaption>
</figure>

We take a Bayesian view of probability, rather than a frequentist one.
This means that we see probabilities as describing our knowledge about
events, rather than tallying up frequencies by which they occur. Think
of the weather-person talking about the probability of rain tomorrow.
Probabilities viewed this way can be used to describe knowledge about
the state of the world, and how actions affect the state of an agent and
the world.

<figure id="fig:Frequentist">
<img src="https://github.com/gtbook/robotics/blob/main/Figures3/N3-Frequentist.png?raw=1"  style="width:9cm" alt="" />
<figcaption>The caricature of the frequentist view involves counting many heads and tails.</figcaption>
</figure>

This is to be contrasted with a frequentist view, where probabilities
are used to describe the frequencies of events in a series of repeated
trials. A Bayesian, instead, might qualify their knowledge about an
event that has not even happened yet, let alone multiple times. Of
course, in most cases this belief is based on experience, i.e., lots of
repeated events in the past, and so it can be seen that perhaps these
views are not so different after all.


## GTSAM 101

Most everything above should by now be familiar, but we used a new `Variables` method, `discrete_series`, to define a time series of state variables. The signature of this method is
```python
def discrete_series(self, character: str, indices: Iterable[int],
                    domain: List[str]) -> Dict[int, DiscreteKey]:
    """Create several discrete variables with Symbol names.

    Args:
        character (str): a single character.
        indices: (Iterable[int]): a set of integer indices.
        domain (List[str]): names for the different values.

    Returns:
        Dict[int, DiscreteKey], i.e., [(gtsam.Key, cardinality)]
    """
```

For example, the following creates a series of 5 state variables:

In [9]:
states = VARIABLES.discrete_series('X', [1, 2, 3], vacuum.rooms)


In [10]:
print(states)


{1: (6341068275337658369, 5), 2: (6341068275337658370, 5), 3: (6341068275337658371, 5)}


When we print the results, we see that we now get a dictionary of DiscreteKeys, i.e., integer tuples of the form *(Key, cardinality)*. However, the "Keys" now seem to be very large integers. This is because for series of variables we use the `gtsam.Symbol` type, composed of a single character and an integer index:

In [11]:
symbol = gtsam.Symbol('X', 1)
print(symbol.string())


X1


GTSAM internally stores symbols as a 64-bit integer key, with the 8-bit character in the most significant bits, which explains the large integer value:

In [12]:
print(symbol.key())


6341068275337658369


You can see that this corresponds to the first state above. However, as before, pretty printing translates these into a nicer looking strings wherever it matters.